<a href="https://colab.research.google.com/github/itonito/Coursera_Capstone/blob/master/Cluster_Map_SegmentingClusteringToronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

from urllib import request
from bs4 import BeautifulSoup

In [0]:
def scraping():
    #url
    url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

    #get html
    html = request.urlopen(url)

    #set BueatifulSoup
    soup = BeautifulSoup(html, "html.parser")

    #get Table    
    table = soup.find_all('table')[0] 
    df = pd.read_html(str(table))

    return df[0]


In [0]:
my_df = scraping()
new_header = my_df.iloc[0] 
my_df = my_df[1:] 
my_df.columns = new_header
my_df = my_df.replace('Not assigned',np.NaN)
my_df = my_df.dropna(subset=['Borough'])
my_df = my_df.reset_index(drop=True)

In [0]:
for i, row in my_df.iterrows():
    if pd.isnull(my_df.at[i,'Neighbourhood']):
        my_df.at[i,'Neighbourhood'] = my_df.at[i,'Borough']

In [0]:
my_df = my_df.groupby(['Postcode','Borough'],as_index=False).agg({'Neighbourhood':', '.join})

In [0]:
ll_df = pd.read_csv('http://cocl.us/Geospatial_data')


In [0]:
new_df = pd.merge(my_df,ll_df, left_on='Postcode', right_on='Postal Code').drop('Postal Code',axis=1)

Install folium using pip install:

In [17]:
!pip install folium=0.5.0
import folium

print('Folium installed and imported!')


Invalid requirement: 'folium=0.5.0'
= is not a valid operator. Did you mean == ?
Folium installed and imported!


In [0]:
# define the canada map
canada_map = folium.Map(location=[43.653908, -79.384293], zoom_start=11)

# Initial folium map creation
This is an initial attempt for creating map of Toronto with all the neighbourhood marked with Red dot.

In [26]:
# instantiate a feature group for the neighbours in the dataframe
neighbours = folium.map.FeatureGroup()

# loop through the neibours and add each to the neighbours feature group
for lat, lng, in zip(new_df.Latitude, new_df.Longitude):
    neighbours.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=8, # define how big you want the circle markers to be
            color='red'
        )
    )

# add neighbours to map
canada_map.add_child(neighbours)

In [0]:
# Get all the locations for York
York_df = new_df.loc[new_df['Borough'] == 'York']

In [0]:
# Get all the locations for Downtown Toronto
DT_df = new_df.loc[new_df['Borough'] == 'Downtown Toronto']

# Second map creation
This time, I will mark the York borough with Green to make it stand out from the rest of the borough.

Notice that York is spread out towards the west side of Toronto.

In [30]:
# instantiate a feature group for the neighbours in the dataframe
York = folium.map.FeatureGroup()

# loop through the neibours and add each to the neighbours feature group
for lat, lng, in zip(York_df.Latitude, York_df.Longitude):
    York.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=8, # define how big you want the circle markers to be
            color='green'
        )
    )

# add neighbours to map
canada_map.add_child(York)

# Third map creation
The third map creation marks Down Town Toronto borough in blue.
Down town Toronto, as its name suggests, are closely clustered around the center of the city near lake Ontario.

In [31]:
# instantiate a feature group for the neighbours in the dataframe
DT = folium.map.FeatureGroup()

# loop through the neibours and add each to the neighbours feature group
for lat, lng, in zip(DT_df .Latitude, DT_df .Longitude):
    DT.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=8, # define how big you want the circle markers to be
            color='blue'
        )
    )

# add neighbours to map
canada_map.add_child(DT)